# Portfolio Stress Test - Data Preprocessing (Clean Version)

This notebook preprocesses position data for stress testing by:
1. Dropping Symbol and Size_USD columns
2. Filling missing Current_Price values using spot prices from same token
3. Using Bybit API (ccxt) for remaining missing prices
4. **Converting Future positions to token units (Size = Size / Current_Price)**
5. Removing rows with missing prices

In [17]:
import pandas as pd
import ccxt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Display options for better output - no scientific notation
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.float_format', '{:.8f}'.format)
np.set_printoptions(suppress=True)

## Step 1: Load Raw Data

In [18]:
# Load the raw data
df = pd.read_csv('output.csv')

print(f"Original data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
display(df.head())

Original data shape: (194, 15)
Columns: ['Timestamp', 'Token', 'Position_Type', 'Symbol', 'Expiry', 'Strike', 'Size', 'Size_USD', 'Option_Type', 'Volatility', 'Interest_Rate', 'Source', 'Current_Price', 'Balance', 'Config_URL']

First few rows:


,Timestamp,Token,Position_Type,Symbol,Expiry,Strike,Size,Size_USD,Option_Type,Volatility,Interest_Rate,Source,Current_Price,Balance,Config_URL
0,2025-09-05T10:19...,OXT,Spot,NaN,NaN,NaN,-3153056.67600000,-168570.29250000,NaN,NaN,NaN,internal,0.05346250,-3153056.67600000,https://docs.goo...
1,2025-09-05T10:19...,OXT,Option,NaN,16/09/2025,0.06369291,5000000.00000000,318464.56950000,Call,0.00000000,0.00000000,options-data-range,0.05346250,-3153056.67600000,https://docs.goo...
2,2025-09-05T10:19...,OXT,Option,NaN,16/09/2025,0.05831982,5000000.00000000,291599.10850000,Call,0.00000000,0.00000000,options-data-range,0.05346250,-3153056.67600000,https://docs.goo...
3,2025-09-05T10:19...,OXT,Option,NaN,16/09/2025,0.05340000,5000000.00000000,267000.00000000,Call,0.00000000,0.00000000,options-data-range,0.05346250,-3153056.67600000,https://docs.goo...
4,2025-09-05T10:19...,OXT,Option,NaN,16/09/2025,0.04889521,5000000.00000000,244476.05610000,Put,0.00000000,0.00000000,options-data-range,0.05346250,-3153056.67600000,https://docs.goo...


## Step 2: Drop Symbol and Size_USD Columns

In [19]:
# Drop Symbol and Size_USD columns as requested
columns_to_drop = ['Symbol', 'Size_USD']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    df = df.drop(columns=existing_columns_to_drop)
    print(f"Dropped columns: {existing_columns_to_drop}")

print(f"New shape: {df.shape}")
print(f"Remaining columns: {list(df.columns)}")

Dropped columns: ['Symbol', 'Size_USD']
New shape: (194, 13)
Remaining columns: ['Timestamp', 'Token', 'Position_Type', 'Expiry', 'Strike', 'Size', 'Option_Type', 'Volatility', 'Interest_Rate', 'Source', 'Current_Price', 'Balance', 'Config_URL']


## Step 3: Analyze Missing Current Prices

In [20]:
# Identify missing Current_Price values
missing_prices_mask = df['Current_Price'].isna() | (df['Current_Price'] == 0) | (df['Current_Price'] == '')
missing_count = missing_prices_mask.sum()

print(f"Found {missing_count:,} rows with missing current prices")

if missing_count > 0:
    print("\nPositions with missing prices:")
    missing_df = df[missing_prices_mask][['Token', 'Position_Type', 'Size', 'Current_Price']]
    display(missing_df)

Found 31 rows with missing current prices

Positions with missing prices:


,Token,Position_Type,Size,Current_Price
163,ETH,Option,200.00000000,NaN
164,ETH,Option,500.00000000,NaN
165,AGI,Future,8476.00000000,NaN
166,BTC,Future,-67545.11036000,NaN
167,CFX,Future,-18654.33050000,NaN
168,ETH,Future,-302999.33020000,NaN
169,ICP,Future,384079.51990000,NaN
170,IOST,Future,47268.41186000,NaN
171,NEAR,Future,645015.65660000,NaN
172,PUMPFUN,Future,41958.00000000,NaN


## Step 4: Fill Missing Prices from Spot Positions

In [21]:
# Create a mapping of token to current price from spot positions
spot_prices = {}
spot_mask = df['Position_Type'].str.upper() == 'SPOT'
valid_price_mask = ~(df['Current_Price'].isna() | (df['Current_Price'] == 0) | (df['Current_Price'] == ''))

spot_with_prices = df[spot_mask & valid_price_mask]
for _, row in spot_with_prices.iterrows():
    token = row['Token']
    price = row['Current_Price']
    if token not in spot_prices and pd.notna(price) and price != 0:
        spot_prices[token] = float(price)

print(f"Found spot prices for {len(spot_prices):,} tokens:")
for token, price in spot_prices.items():
    print(f"  {token}: ${price:,.8f}")

Found spot prices for 18 tokens:
  OXT: $0.05346250
  FET: $0.60947000
  ICP: $4.79958333
  ZEN: $7.15610000
  CSPR: $0.00969925
  ACX: $0.18200071
  IOST: $0.00325250
  SAFE: $0.42228040
  NEAR: $2.42244375
  CELO: $0.29480000
  CFX: $0.17231500
  VET: $0.02375000
  MINA: $0.18030000
  RARE: $0.05454900
  AGI: $0.04227750
  TNSR: $0.11446250
  ETH: $4,404.07400400
  BTC: $112,194.99590000


In [22]:
# Fill missing prices using spot prices
filled_from_spot = 0
for idx, row in df[missing_prices_mask].iterrows():
    token = row['Token']
    if token in spot_prices:
        df.at[idx, 'Current_Price'] = spot_prices[token]
        filled_from_spot += 1

print(f"Filled {filled_from_spot:,} prices from spot positions")

Filled 23 prices from spot positions


## Step 5: Use Bybit API for Remaining Missing Prices

In [23]:
# Check for still missing prices
still_missing_mask = df['Current_Price'].isna() | (df['Current_Price'] == 0) | (df['Current_Price'] == '')
still_missing_count = still_missing_mask.sum()

print(f"Still missing prices for {still_missing_count:,} positions")

if still_missing_count > 0:
    print("\nTokens still needing prices:")
    tokens_needing_prices = df[still_missing_mask]['Token'].unique()
    print(tokens_needing_prices)

Still missing prices for 8 positions

Tokens still needing prices:
['PUMPFUN' 'XRP' 'HOME' 'LINEA' 'SOL' 'SUI' 'TOWNS']


In [24]:
# Fetch prices from Bybit using ccxt
if still_missing_count > 0:
    try:
        exchange = ccxt.bybit({
            'sandbox': False,  # Use live API
        })
        
        bybit_prices = {}
        tokens_needing_prices = df[still_missing_mask]['Token'].unique()
        
        for token in tokens_needing_prices:
            try:
                symbol = f"{token}/USDT"
                ticker = exchange.fetch_ticker(symbol)
                price = ticker['last']
                bybit_prices[token] = price
                print(f"✅ Fetched {token}: ${price:,.8f}")
            except Exception as e:
                print(f"❌ Could not fetch price for {token}: {str(e)}")
        
        # Fill remaining missing prices with Bybit data
        filled_from_bybit = 0
        for idx, row in df[still_missing_mask].iterrows():
            token = row['Token']
            if token in bybit_prices:
                df.at[idx, 'Current_Price'] = bybit_prices[token]
                filled_from_bybit += 1
        
        print(f"\nFilled {filled_from_bybit:,} prices from Bybit")
        
    except Exception as e:
        print(f"Error initializing Bybit exchange: {str(e)}")
        print("Continuing with available price data...")

❌ Could not fetch price for PUMPFUN: bybit does not have market symbol PUMPFUN/USDT
✅ Fetched XRP: $2.84360000
✅ Fetched HOME: $0.04032000
❌ Could not fetch price for LINEA: bybit does not have market symbol LINEA/USDT
✅ Fetched SOL: $206.34000000
✅ Fetched SUI: $3.38650000
✅ Fetched TOWNS: $0.02330000

Filled 6 prices from Bybit


## Step 6: Convert Future Positions to Token Units

In [25]:
# Convert Future positions from USD notional to token units
# For futures: Size = Size / Current_Price (convert from USD to token units)

print("Converting Future positions to token units...")

# Show current futures positions before conversion
futures_mask = df['Position_Type'].str.upper() == 'FUTURE'
futures_count = futures_mask.sum()

print(f"Found {futures_count:,} Future positions to convert")

if futures_count > 0:
    print("\nFuture positions before conversion:")
    futures_before = df[futures_mask][['Token', 'Size', 'Current_Price']].head(10)
    for _, row in futures_before.iterrows():
        print(f"  {row['Token']}: Size=${row['Size']:,.0f} USD, Price=${row['Current_Price']:,.6f}")
    
    # Convert Future position sizes from USD to token units
    converted_count = 0
    for idx, row in df[futures_mask].iterrows():
        if pd.notna(row['Current_Price']) and row['Current_Price'] != 0:
            original_size = row['Size']
            new_size = original_size / row['Current_Price']
            df.at[idx, 'Size'] = new_size
            converted_count += 1
    
    print(f"\n✅ Converted {converted_count:,} Future positions to token units")
    
    print("\nFuture positions after conversion:")
    futures_after = df[futures_mask][['Token', 'Size', 'Current_Price']].head(10)
    for _, row in futures_after.iterrows():
        print(f"  {row['Token']}: Size={row['Size']:,.6f} tokens, Price=${row['Current_Price']:,.6f}")
else:
    print("No Future positions found to convert")

print(f"\nFuture conversion complete!")

Converting Future positions to token units...
Found 29 Future positions to convert

Future positions before conversion:
  AGI: Size=$8,476 USD, Price=$0.042278
  BTC: Size=$-67,545 USD, Price=$112,194.995900
  CFX: Size=$-18,654 USD, Price=$0.172315
  ETH: Size=$-302,999 USD, Price=$4,404.074004
  ICP: Size=$384,080 USD, Price=$4.799583
  IOST: Size=$47,268 USD, Price=$0.003253
  NEAR: Size=$645,016 USD, Price=$2.422444
  PUMPFUN: Size=$41,958 USD, Price=$nan
  SAFE: Size=$51,617 USD, Price=$0.422280
  VET: Size=$47,500 USD, Price=$0.023750

✅ Converted 27 Future positions to token units

Future positions after conversion:
  AGI: Size=200,484.891491 tokens, Price=$0.042278
  BTC: Size=-0.602033 tokens, Price=$112,194.995900
  CFX: Size=-108,257.148246 tokens, Price=$0.172315
  ETH: Size=-68.799782 tokens, Price=$4,404.074004
  ICP: Size=80,023.513137 tokens, Price=$4.799583
  IOST: Size=14,532,947.535742 tokens, Price=$0.003253
  NEAR: Size=266,266.515621 tokens, Price=$2.422444
  PUMP

## Step 7: Remove Rows with Missing Prices

In [26]:
# Drop rows where we couldn't fetch prices
rows_before = len(df)
missing_price_mask = df['Current_Price'].isna() | (df['Current_Price'] == 0) | (df['Current_Price'] == '')

if missing_price_mask.any():
    print(f"Dropping {missing_price_mask.sum():,} rows with missing prices:")
    missing_tokens = df[missing_price_mask][['Token', 'Position_Type', 'Size']]
    display(missing_tokens)
    
    # Drop the rows
    df = df[~missing_price_mask].copy()
    
    rows_after = len(df)
    print(f"\nRows before: {rows_before:,}")
    print(f"Rows after: {rows_after:,}")
    print(f"Dropped: {rows_before - rows_after:,} rows")
else:
    print("No rows with missing prices to drop")

Dropping 2 rows with missing prices:


,Token,Position_Type,Size
172,PUMPFUN,Future,41958.00000000
186,LINEA,Future,16328.40000000



Rows before: 194
Rows after: 192
Dropped: 2 rows


## Step 8: Final Data Cleanup and Validation

In [27]:
# Final data validation and cleanup
df['Current_Price'] = pd.to_numeric(df['Current_Price'], errors='coerce')
df['Size'] = pd.to_numeric(df['Size'], errors='coerce')

# Remove any rows that still have invalid data
valid_data_mask = ~(df['Current_Price'].isna() | df['Size'].isna())
invalid_count = (~valid_data_mask).sum()

if invalid_count > 0:
    print(f"Removing {invalid_count:,} rows with invalid numeric data")
    df = df[valid_data_mask].copy()

print(f"Final preprocessing complete!")
print(f"Clean dataset shape: {df.shape}")
print(f"All positions have valid prices and sizes: {(~df['Current_Price'].isna()).all() and (~df['Size'].isna()).all()}")

Final preprocessing complete!
Clean dataset shape: (192, 13)
All positions have valid prices and sizes: True


## Step 9: Create Position Summary

In [28]:
# Calculate position values
df['Position_Value_USD'] = df['Size'] * df['Current_Price']

# Create position summary
print("=" * 60)
print("POSITION SUMMARY")
print("=" * 60)

# Group by Token and Position_Type
summary = df.groupby(['Token', 'Position_Type']).agg({
    'Size': 'sum',
    'Current_Price': 'first',
    'Balance': 'first',
    'Position_Value_USD': 'sum'
}).reset_index()

# Sort by absolute position value
summary['Abs_Value'] = summary['Position_Value_USD'].abs()
summary = summary.sort_values('Abs_Value', ascending=False)

print("\nTop 15 Positions by Value:")
display(summary.head(15)[['Token', 'Position_Type', 'Size', 'Current_Price', 'Position_Value_USD']])

POSITION SUMMARY

Top 15 Positions by Value:


,Token,Position_Type,Size,Current_Price,Position_Value_USD
24,ICP,Option,1300000.00000000,4.79958333,6239458.33290000
32,NEAR,Option,1550000.00000000,2.42244375,3754787.81250000
17,ETH,Option,700.00000000,4404.07400400,3082851.80280000
25,ICP,Spot,-619812.98910000,4.79958333,-2974844.09206127
23,ICP,Future,414944.82535741,4.79958333,1991562.26790000
34,OXT,Option,25000000.00000000,0.05346250,1336562.50000000
16,ETH,Future,-242.64601623,4404.07400400,-1068631.01226000
20,FET,Option,1650000.00000000,0.60947000,1005625.50000000
27,IOST,Option,300000000.00000000,0.00325250,975750.00000000
6,BTC,Future,-8.53600612,112194.99590000,-957697.17146000


In [29]:
# Display all ICP positions
icp_positions = df[df['Token'] == 'ETH']
print(f"Found {len(icp_positions)} ICP positions:")
print("\nAll ICP positions:")
display(icp_positions[['Token', 'Position_Type', 'Size', 'Current_Price', 'Position_Value_USD']])

Found 6 ICP positions:

All ICP positions:


,Token,Position_Type,Size,Current_Price,Position_Value_USD
161,ETH,Spot,211.18684100,4404.07400400,930082.47643498
163,ETH,Option,200.00000000,4404.07400400,880814.80080000
164,ETH,Option,500.00000000,4404.07400400,2202037.00200000
168,ETH,Future,-68.79978173,4404.07400400,-302999.33020000
177,ETH,Future,12.85012615,4404.07400400,56592.90654000
182,ETH,Future,-186.69636065,4404.07400400,-822224.58860000


In [30]:
# Summary by token (total exposure)
token_summary = df.groupby('Token').agg({
    'Size': 'sum',
    'Current_Price': 'first',
    'Position_Value_USD': 'sum'
}).reset_index()

token_summary['Total_Exposure_USD'] = token_summary['Position_Value_USD']
token_summary = token_summary.sort_values('Total_Exposure_USD', key=abs, ascending=False)

print("\nTotal Exposure by Token:")
display(token_summary[['Token', 'Total_Exposure_USD']].head(15))


Total Exposure by Token:


,Token,Total_Exposure_USD
9,ICP,5256176.50873873
12,NEAR,3435168.05515041
6,ETH,2944303.26697498
13,OXT,1167992.20745935
7,FET,1024732.29392623
15,SAFE,964000.18108181
0,ACX,917633.67294534
20,VET,852223.35717875
10,IOST,742155.10831102
18,TNSR,640507.13582310


## Step 10: Save Processed Data

In [31]:
# Save processed data with proper formatting
output_file = 'processed_positions.csv'
df.to_csv(output_file, index=False, float_format='%.8f')
print(f"Processed data saved to: {output_file}")

print(f"\nFinal dataset info:")
print(f"- {len(df):,} total positions")
print(f"- {df['Token'].nunique():,} unique tokens")
print(f"- {df['Position_Type'].nunique():,} position types: {list(df['Position_Type'].unique())}")
print(f"- Total portfolio value: ${df['Position_Value_USD'].sum():,.2f}")

# Position type breakdown
pos_type_counts = df['Position_Type'].value_counts()
print(f"\nPosition breakdown:")
for pos_type, count in pos_type_counts.items():
    print(f"- {pos_type}: {count:,} positions")

Processed data saved to: processed_positions.csv

Final dataset info:
- 192 total positions
- 23 unique tokens
- 3 position types: ['Spot', 'Option', 'Future']
- Total portfolio value: $20,760,216.31

Position breakdown:
- Option: 147 positions
- Future: 27 positions
- Spot: 18 positions


## Data Quality Check

In [32]:
# Final data quality check with proper formatting
print("Data Quality Summary:")
print("-" * 40)
print(f"Rows with valid prices: {(~df['Current_Price'].isna()).sum():,}")
print(f"Rows with zero positions: {(df['Size'] == 0).sum():,}")
print(f"Unique tokens: {df['Token'].nunique():,}")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")

# Show sample of final data
print("\nSample of processed data (first 10 rows):")
sample_cols = ['Token', 'Position_Type', 'Size', 'Current_Price', 'Position_Value_USD']
display(df[sample_cols].head(10))

print("\n✅ Preprocessing complete! Ready for delta calculations.")

Data Quality Summary:
----------------------------------------
Rows with valid prices: 192
Rows with zero positions: 73
Unique tokens: 23
Date range: 2025-09-05T10:19:51.740Z to 2025-09-05T10:19:51.740Z

Sample of processed data (first 10 rows):


,Token,Position_Type,Size,Current_Price,Position_Value_USD
0,OXT,Spot,-3153056.67600000,0.05346250,-168570.29254065
1,OXT,Option,5000000.00000000,0.05346250,267312.50000000
2,OXT,Option,5000000.00000000,0.05346250,267312.50000000
3,OXT,Option,5000000.00000000,0.05346250,267312.50000000
4,OXT,Option,5000000.00000000,0.05346250,267312.50000000
5,OXT,Option,5000000.00000000,0.05346250,267312.50000000
6,FET,Spot,-490567.83250000,0.60947000,-298986.37687377
7,FET,Option,0.00000000,0.60947000,0.00000000
8,FET,Option,150000.00000000,0.60947000,91420.50000000
9,FET,Option,250000.00000000,0.60947000,152367.50000000



✅ Preprocessing complete! Ready for delta calculations.
